In [17]:
#!py -m pip install pyodbc pandas db-dtypes
#!py -m pip install google-cloud-bigquery google-auth google-auth-oauthlib pandas_gbq

In [18]:
import pyodbc
import pandas as pd
import pandas_gbq
import json
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

In [19]:
# Open and read the JSON file
with open('config/mom_db_config.json', 'r') as file:
    mom_db_config = json.load(file)

In [20]:
MOM_CONN_STR = (
    f"{mom_db_config['driver']}"
    f"{mom_db_config['server']}"
    f"{mom_db_config['database']}"
    f"{mom_db_config['user']}"
    f"{mom_db_config['password']}"
)
PROJECT_ID = "sturdy-tome-468208-k2"
DATASET_ID = "dev"
TABLE_IDS = {
    "orders" : "orders"
}

In [21]:
# Create connection
mom_conn = pyodbc.connect(MOM_CONN_STR)

In [22]:
# Initialize client (uses default credentials)
bq_client = bigquery.Client(project=PROJECT_ID)

In [ ]:
# Get the latest order id
query = f"""
    SELECT MAX(order_id) AS max_id FROM `{DATASET_ID}.{TABLE_IDS['orders']}` 
"""
max_id = None
try :
    # Run query and return results as a DataFrame
    df = bq_client.query(query).to_dataframe()
    if len(df) > 0 :
        max_id = df.iloc(0)['max_id']
except NotFound:
    print(f"Table not found: {TABLE_IDS['orders']}")

SyntaxError: invalid syntax (2402438273.py, line 4)

NameError: name 'df' is not defined

In [ ]:
last_order_time  = "2025-08-06 12:27:05.000"
last_order_num = 1588550

params = []
parms_str = ""
if max_id is not None:
    params = [max_id]
    parms_str = " WHERE orderno > ? "

query = f"""
    SELECT
        orderno AS order_id,
        custnum AS customer_id,
        entrytime AS entry_time,
        ord_total AS order_total
        FROM CMS 
        {parms_str}
        ORDER BY order_id ASC
"""

df = pd.read_sql(query, mom_conn, params = params)

C:\Users\danny.sloan\AppData\Local\Temp\ipykernel_22876\47242132.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mom_conn)#, params=[last_order_num])


In [ ]:
df.shape

(1510060, 4)

In [ ]:
df.tail(10)

,order_id,customer_id,entry_time,order_total
1510050,1588616.0,323786.0,2025-08-06 15:08:39,90.0
1510051,1588617.0,381457.0,2025-08-06 15:08:50,35.0
1510052,1588618.0,935854.0,2025-08-06 15:09:15,35.0
1510053,1588619.0,802832.0,2025-08-06 15:10:20,85.0
1510054,1588620.0,935855.0,2025-08-06 15:13:44,60.0
1510055,1588621.0,258255.0,2025-08-06 15:20:36,41.0
1510056,1588622.0,735520.0,2025-08-06 15:23:35,60.0
1510057,1588623.0,372708.0,NaT,100.0
1510058,1588624.0,935598.0,2025-08-06 15:23:51,135.0
1510059,1588625.0,880630.0,2025-08-06 15:25:19,55.0


In [ ]:
# We need to fix the null entrytimes, just backwards fill from the previous seen entrytime
df['entry_time'] = pd.to_datetime(df['entry_time'], errors='coerce')  # ensure datetime type
df['entry_time'] = df['entry_time'].fillna(method='ffill')  # forward-fill

C:\Users\danny.sloan\AppData\Local\Temp\ipykernel_22876\1582709845.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['entry_time'] = df['entry_time'].fillna(method='ffill')  # forward-fill


In [ ]:
df.tail(10)

,order_id,customer_id,entry_time,order_total
1510050,1588616.0,323786.0,2025-08-06 15:08:39,90.0
1510051,1588617.0,381457.0,2025-08-06 15:08:50,35.0
1510052,1588618.0,935854.0,2025-08-06 15:09:15,35.0
1510053,1588619.0,802832.0,2025-08-06 15:10:20,85.0
1510054,1588620.0,935855.0,2025-08-06 15:13:44,60.0
1510055,1588621.0,258255.0,2025-08-06 15:20:36,41.0
1510056,1588622.0,735520.0,2025-08-06 15:23:35,60.0
1510057,1588623.0,372708.0,2025-08-06 15:23:35,100.0
1510058,1588624.0,935598.0,2025-08-06 15:23:51,135.0
1510059,1588625.0,880630.0,2025-08-06 15:25:19,55.0


In [ ]:
mom_conn.close()

In [ ]:
#!gcloud auth application-default login

In [ ]:
# Configure load job — let schema be auto-detected
job_config = bigquery.LoadJobConfig(
    autodetect = True,              # This makes schema generation automatic
    write_disposition = "WRITE_APPEND"  # Or WRITE_TRUNCATE / WRITE_EMPTY
)

# Load DataFrame into BigQuery
job = bq_client.load_table_from_dataframe(
    df,
    f"{DATASET_ID}.{TABLE_IDS['orders']}",
    job_config=job_config
)
job.result()  # Wait for the load to complete

print("Loaded {} rows into {}".format(df.shape[0], TABLE_IDS['orders']))

Loaded 1510060 rows into orders


: 

In [ ]:
query = f"""
    SELECT * FROM `{DATASET_ID}.{TABLE_IDS['orders']}`
"""

# Run query and return results as a DataFrame
df = bq_client.query(query).to_dataframe()

c:\Users\danny.sloan\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:
df.shape

(1510042, 5)

In [ ]:
df.head(10)

,order_id,customer_id,entry_time,order_total,check_amount
0,2.0,42850.0,2004-03-31 12:52:53,4.00,4.00
1,4.0,42850.0,2004-03-31 15:43:24,11.75,11.75
2,12.0,42873.0,2004-04-01 14:50:33,41.97,41.97
3,14.0,42874.0,2004-04-01 15:52:40,26.97,26.97
4,15.0,42875.0,2004-04-01 16:10:36,26.97,26.97
5,23.0,42886.0,2004-04-02 10:23:12,17.98,17.98
6,26.0,42888.0,2004-04-02 11:45:34,19.99,19.99
7,35.0,42895.0,2004-04-02 14:15:04,26.97,26.97
8,37.0,42898.0,2004-04-02 16:08:21,26.97,26.97
9,38.0,42904.0,2004-04-05 12:09:29,31.98,31.98
